Damgard-Jurik Scheme of Paillier
================

## Generalisation

The publickey cryptosystem uses computations modulo $n^{s+1}$ where $n$ is an $RSA$ modulus and $s$ is a natural number. It contains **Paillier's scheme** as a special case by setting $s=1$

We start from the observation that if $n=pq,p,q$ odd primes, then $Z^*_{n^{s+1}}$ as a multiplicative group is a direct product $G\times H$, where $G$ ic cyclic of order $n^s$ and $H$ is isomorphic to $Z^*_{n}$, which follows directly from elementary number theory.

Thus, the factor group $\bar{G}=Z*_{n^{s+1}}/H$ is also cyclic of order $n^s$. For an arbitary element $a \in Z_n^{s+1}$, we let $\bar{a}  H$ we denote the element represented by $a$ in the factor group $\bar{G}$.

* Lemma 1: For any $s< p, q$, the element $n+1$ has order $n^s$ in $Z_n^{s+1}$.

In [1]:
from klefki.types.algebra.meta import field
from klefki.types.algebra.utils import randfield

In [2]:
from klefki.numbers.primes import generate_prime

k = 32
p, q = generate_prime(32), generate_prime(32)
s = 9

n = p * q
j = generate_prime(k)

G = field(n**s)
MG = field(n ** (s+1))
H = field(n)

### Key Generation

On input the security parameter $k$, choose an $RSA$ modulus $n=pq$ of length $k$ bits. Also choose an element $g\in Z_n^{s+1}$ such that $g=(1+n)^j$ mod $n^{s+1}$ for a known $j$ relatively prime to $n$ and $x \in H$.

This can be done, $e.g$, by choosing $j, x$ as random first and computing $g$;

Let $\lambda$ be the $lcm$ of $p-1$ and $q-1$. By the Chinese Remainder Theorem, choose $d$ such that $d$ mod $n\in Z_n$ and $d=0$ mod $\lambda$. 

Any such choice of $d$ will work in the following.

Now the pubkey is $n, g$ while the secret key is $d$.

In [3]:
from klefki.numbers.primes import generate_prime
from klefki.numbers import lcm
from random import randint
from math import gcd

In [4]:
j = generate_prime(32)
assert gcd(j, n) == 1
x = MG(randfield(H))
g = MG(1 + n) ** j * x

In [5]:
lam = lcm(p-1, q-1)
LF = field(lam)

In [6]:
#d = randint(1, int(n / lam)) * lam
d = lam
assert d < n
assert d % lam == 0

pubkey = (n, g)
privkey = d

### Encryption

The paintext set is $Z_{n^s}$. Given a paintext $i$, choose a random $r \leftarrow Z^*_{n^{s+1}}$, and lte the cipertext be $E(i, r) = g^ir^{n^s} \mod n^{s+1}$

In [7]:
r = randfield(MG)
i = randfield(G).value
e = lambda i, r: g ** i * r ** (n**s)

In [8]:
c = e(i, r)

### Decryption

Given a ciphertext $c$, first compute $c^d \mod n^{s+1}$, Clearly, if $c=E(v, r)$, we get

\begin{align}
c^d &= (g^i r^{n^s})^d\\
&=((1+n)^{ij} x^i r^{n^s})^d\\
&=(1+n)^{jid \mod n^s}(x^ir^{n^s})^{d \mod \lambda}\\
&=(1+n)^{jid \mod n^s}
\end{align}


In [9]:
c ** d ==  (g ** i * r ** (n**s))**d \
       == ((MG(1 + n) ** j * x) ** i * r ** (n**s))**d \
       == (MG(1 + n) ** (j * i) * x ** i * r ** (n**s))**d \
       == MG(1 + n) ** (j * i * d) * (x ** i * r ** (n**s)) ** d

True

## Ref:
    
    * I. Damg˚ard and M. Jurik. A generalisation, a simpliﬁcation and some applications of paillier’s proba- bilistic public-key system. In Public Key Cryptography, pages 119–136, 2001.